In [11]:
import pandas as pd
df = pd.read_csv('../data/data_segmented.csv',index_col=0)

In [12]:
df.iloc[:,-1].replace(['GAIT', 'RAMP', 'STEP', 'UNEVEN'],
                        [0, 1,2,3], inplace=True)

In [13]:
X = df.iloc[:,0:-1]
y = df.iloc[:,-1]

In [ ]:
print(X.shape)
print(y.shape)

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import classification_report

h = 0.02  # step size in the mesh

names = [
    "Gradient Boosting",
    "Nearest Neighbors",
    "Linear SVM",
    "RBF SVM",
    "Gaussian Process",
    "Decision Tree",
    "Random Forest",
    "Neural Net",
    "AdaBoost",
    "Naive Bayes",
    "QDA",
]

classifiers = [
    GradientBoostingClassifier(),
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
    GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1, max_iter=1000),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis(),
]

def training(train, test, fold_no, model):
    x_train = train.drop(['117'],axis=1)
    y_train = train.iloc[:,-1]
    x_test = test.drop(['117'],axis=1)
    y_test = test.iloc[:,-1]
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    plot_confusion_matrix(model, x_train,y_train)
    print(classification_report(y_test,y_pred,digits = 4))

for name, clf in zip(names, classifiers):
    print(name)
    skf = StratifiedKFold(n_splits=10)
    fold_no = 1
    for train_index,test_index in skf.split(X, y):
        train = df.iloc[train_index,:]
        test = df.iloc[test_index,:]
        training(train, test, fold_no, clf)
        fold_no += 1